In [3]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, ConfigurableField 
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [4]:
# Default model choices
MODEL_CHOICES = ["gpt-4o", "gpt-4o-mini", "o3-mini"]
DEFAULT_MODEL = os.environ.get("OPENAI_MODEL_NAME", "gpt-4o")

# LangChain Implementation


def create_chain(temperature, basemodel):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI assistant that provides detailed and accurate responses."),
        ("human", "{input}")
    ])
    model = ChatOpenAI(
        temperature=temperature,
        model=basemodel,
        streaming=True
    )
    chain = (
        {"input": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    return chain

In [5]:
# Chat history


def respond(message, model_type, temperature, top_k, basemodel, history):
    history = history or []
    os.environ["OPENAI_MODEL_NAME"] = basemodel
    chain = create_chain(temperature, basemodel)
    full_response = ""
    for chunk in chain.stream(message):
        full_response += chunk
        yield "", history + [(message, full_response + "▌")]
    yield "", history + [(message, full_response)]

In [8]:
# Create Gradio interface
with gr.Blocks(title="LLM Interface Test") as demo:
    gr.Markdown("# LLM Interface Test (LangChain Only)")

    with gr.Row():
        with gr.Column():
            model_type = gr.Radio(
                choices=["test1", "test2", "test3"],
                label="Select Model Type",
                value="test1"
            )
            temperature = gr.Slider(
                minimum=0.0,
                maximum=2.0,
                value=0.7,
                step=0.05,
                label="Temperature"
            )
            top_k = gr.Slider(
                minimum=1,
                maximum=100,
                value=40,
                step=1,
                label="Top-k (for sampling, if supported)"
            )
            basemodel = gr.Dropdown(
                choices=MODEL_CHOICES,
                value=DEFAULT_MODEL if DEFAULT_MODEL in MODEL_CHOICES else MODEL_CHOICES[0],
                label="Base Model (OPENAI_MODEL_NAME)"
            )

    chatbot = gr.Chatbot(height=600)
    with gr.Row():
        msg = gr.Textbox(
            show_label=False,
            placeholder="Enter your message here",
            container=False
        )
        submit = gr.Button("Send")

    submit.click(respond, [msg, model_type, temperature, top_k, basemodel, chatbot], [
                 msg, chatbot], api_name="chat")
    msg.submit(respond, [msg, model_type, temperature, top_k,
               basemodel, chatbot], [msg, chatbot], api_name="chat")

if __name__ == "__main__":
    demo.queue().launch(share=True)


/var/folders/yy/yc520yv550j7_tlzc44qpnsr0000gn/T/ipykernel_24236/2284910739.py:32: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=600)


* Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
